# sub003
g1 - g4をまとめてps

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
from math import * 

In [2]:
g1 = 'g1_v002'
g2 = 'g2_v002'
g3 = 'g3_v002'
g4 = 'g4_v002'
g5 = 'g5_v001'

In [3]:
gs = [g1, g2, g3, g4]

In [4]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [5]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [6]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [7]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [8]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)
    
def vis(df, phone):
    gt = get_ground_truth()
    gt['phone'] = gt['collectionName'] + '_' + gt['phoneName']
    tmp1 = df[df['phone']==phone].copy()
    tmp2 = gt[gt['phone']==phone].copy()
    
    tmp1['phoneName'] = 'train'
    tmp2['phoneName'] = 'gt'
    
    tmp = pd.concat([tmp1, tmp2])
    tmp['collectionName'] = 'all'
    
    lat_center = tmp['latDeg'].mean()
    lng_center = tmp['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
        
    visualize_trafic(tmp, center)

In [9]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [10]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [11]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [12]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = get_ground_truth()
    return base_train, base_test, sample_sub, ground_truth

In [13]:
sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
gt = get_ground_truth()
train = pd.DataFrame()
test = pd.DataFrame()

for g in gs:
    train = train.append(pd.read_csv(f'../output/{g}/{g}_train.csv'))
    test = test.append(pd.read_csv(f'../output/{g}/{g}_test.csv'))

# position shift

In [14]:
def WGS84_to_ECEF(lat, lon, alt):
    # convert to radians
    rad_lat = lat * (np.pi / 180.0)
    rad_lon = lon * (np.pi / 180.0)
    a    = 6378137.0
    # f is the flattening factor
    finv = 298.257223563
    f = 1 / finv   
    # e is the eccentricity
    e2 = 1 - (1 - f) * (1 - f)    
    # N is the radius of curvature in the prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
    x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
    y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
    z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
    return x, y, z

transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)

def ECEF_to_WGS84(x,y,z):
    lon, lat, alt = transformer.transform(x,y,z,radians=False)
    return lon, lat, alt
msge = 'millisSinceGpsEpoch'

In [15]:
def position_shift(df,a):

    d = df.copy()
    d['heightAboveWgs84EllipsoidM'] = 63.5
    d['x'], d['y'], d['z'] = zip(*d.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, x.heightAboveWgs84EllipsoidM), axis=1))

    #a = -0.2
    d.sort_values(['phone', msge], inplace=True)
    for fi in ['x','y','z']:
        d[[fi+'p']] = d[fi].shift().where(d['phone'].eq(d['phone'].shift()))
        d[[fi+'diff']] = d[fi]-d[fi+'p']
    #d[['yp']] = d['y'].shift().where(d['phone'].eq(d['phone'].shift()))
    d[['dist']] = np.sqrt(d['xdiff']**2 + d['ydiff']**2+ d['zdiff']**2)
    for fi in ['x','y','z']:
        d[[fi+'new']] = d[fi+'p'] + d[fi+'diff']*(1-a/d['dist'])
    lng, lat, alt = ECEF_to_WGS84(d['xnew'].values,d['ynew'].values,d['znew'].values)
    
    lng[np.isnan(lng)] = d.loc[np.isnan(lng),'lngDeg']
    lat[np.isnan(lat)] = d.loc[np.isnan(lat),'latDeg']
    d['latDeg'] = lat
    d['lngDeg'] = lng
    
    return d 

def objective(trial):
    a = trial.suggest_uniform('a', -1, 1)
    score = get_train_score(position_shift(train, a),gt)
    return score

In [16]:
study = optuna.create_study()
study.optimize(objective, n_trials=30)
opt_a = study.best_params['a']
print(opt_a)

[I 2021-07-10 15:03:16,068] A new study created in memory with name: no-name-96ed3b01-bccd-4967-8df4-c55248a000bb
[I 2021-07-10 15:03:19,473] Trial 0 finished with value: 2.8055514817075213 and parameters: {'a': 0.23884403852013025}. Best is trial 0 with value: 2.8055514817075213.
[I 2021-07-10 15:03:22,750] Trial 1 finished with value: 2.7886953503761003 and parameters: {'a': 0.6520351240327871}. Best is trial 1 with value: 2.7886953503761003.
[I 2021-07-10 15:03:26,107] Trial 2 finished with value: 3.26688136941588 and parameters: {'a': -0.9941541816120714}. Best is trial 1 with value: 2.7886953503761003.
[I 2021-07-10 15:03:29,494] Trial 3 finished with value: 2.78926543318993 and parameters: {'a': 0.6613772429266112}. Best is trial 1 with value: 2.7886953503761003.
[I 2021-07-10 15:03:32,848] Trial 4 finished with value: 3.0930582218676306 and parameters: {'a': -0.6567650809537631}. Best is trial 1 with value: 2.7886953503761003.
[I 2021-07-10 15:03:36,162] Trial 5 finished with va

0.5226213293510277


In [17]:
train = position_shift(train, opt_a)

In [18]:
g5_train = pd.read_csv(f'../output/{g5}/{g5}_train.csv')
train = train.append(g5_train)

In [19]:
result = train_result(train)
print(result.score)
display(result.err)

3.110312769989652


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.165592,2.330394,1.747993
2020-05-14-US-MTV-1_Pixel4XLModded,1.055572,2.187216,1.621394
2020-05-14-US-MTV-2_Pixel4,1.687255,3.078255,2.382755
2020-05-14-US-MTV-2_Pixel4XLModded,1.791188,4.586148,3.188668
2020-05-21-US-MTV-1_Pixel4,1.757794,3.154850,2.456322
2020-05-21-US-MTV-2_Pixel4,0.646381,2.254191,1.450286
2020-05-21-US-MTV-2_Pixel4XL,0.598279,2.149450,1.373864
2020-05-29-US-MTV-1_Pixel4,2.068052,3.490512,2.779282
2020-05-29-US-MTV-1_Pixel4XL,1.848983,3.098770,2.473877


In [20]:
test = position_shift(test, opt_a)

In [21]:
g5_test = pd.read_csv(f'../output/{g5}/{g5}_test.csv')
test = test.append(g5_test)

In [22]:
del sub['latDeg']
del sub['lngDeg']

In [23]:
sub = sub.merge(test[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['phone', 'millisSinceGpsEpoch'], how='left')

In [24]:
sub.to_csv(f'{OUTPUT}/{nb_name}_sub.csv', index=False)

In [33]:
g5c = ['2021-04-22-US-SJC-1', '2021-04-28-US-SJC-1', '2021-04-29-US-SJC-2', 
      '2021-04-22-US-SJC-2', '2021-04-29-US-SJC-3']

In [34]:
exp107 = pd.read_csv('../output/exp045/train_ro_rog_kf_sp0_pm_ps.csv')
exp107_tmp = pd.read_csv('../output/exp107/train_ro_rog_kf_sp0_pm_ps.csv')

In [36]:
exp107 = exp107[~exp107['collectionName'].isin(g5c)]

In [37]:
exp107 = exp107.append(exp107_tmp)

In [38]:
exp107

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,phone,heightAboveWgs84EllipsoidM,x,y,z,xp,xdiff,yp,ydiff,zp,zdiff,dist,xnew,ynew,znew
0,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.423582,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.5,-2.694608e+06,-4.296551e+06,3.854871e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423582,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.5,-2.694608e+06,-4.296551e+06,3.854871e+06,-2.694608e+06,0.000000e+00,-4.296551e+06,0.000000e+00,3.854871e+06,0.000000e+00,0.000000e+00,NaN,NaN,NaN
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423582,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.5,-2.694608e+06,-4.296551e+06,3.854871e+06,-2.694608e+06,0.000000e+00,-4.296551e+06,0.000000e+00,3.854871e+06,0.000000e+00,0.000000e+00,NaN,NaN,NaN
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423582,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.5,-2.694608e+06,-4.296551e+06,3.854871e+06,-2.694608e+06,0.000000e+00,-4.296551e+06,0.000000e+00,3.854871e+06,0.000000e+00,0.000000e+00,NaN,NaN,NaN
4,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.423583,-122.094127,2020-05-14-US-MTV-1_Pixel4,63.5,-2.694608e+06,-4.296552e+06,3.854870e+06,-2.694608e+06,-2.432416e-01,-4.296551e+06,-1.910472e-01,3.854871e+06,-3.804025e-01,4.902770e-01,-2.694608e+06,-4.296551e+06,3.854871e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14508,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760315000,37.334464,-121.899601,2021-04-29-US-SJC-2_SamsungS20Ultra,63.5,-2.683179e+06,-4.310772e+06,3.847011e+06,-2.683179e+06,-1.862645e-09,-4.310772e+06,9.313226e-10,3.847011e+06,0.000000e+00,2.082501e-09,-2.683178e+06,-4.310773e+06,3.847011e+06
14509,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760316000,37.334463,-121.899612,2021-04-29-US-SJC-2_SamsungS20Ultra,63.5,-2.683179e+06,-4.310772e+06,3.847011e+06,-2.683179e+06,0.000000e+00,-4.310772e+06,0.000000e+00,3.847011e+06,0.000000e+00,0.000000e+00,NaN,NaN,NaN
14510,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760317000,37.334465,-121.899601,2021-04-29-US-SJC-2_SamsungS20Ultra,63.5,-2.683179e+06,-4.310772e+06,3.847011e+06,-2.683179e+06,-1.862645e-09,-4.310772e+06,9.313226e-10,3.847011e+06,-4.656613e-10,2.133928e-09,-2.683178e+06,-4.310773e+06,3.847011e+06
14511,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760318000,37.334470,-121.899612,2021-04-29-US-SJC-2_SamsungS20Ultra,63.5,-2.683179e+06,-4.310772e+06,3.847011e+06,-2.683179e+06,0.000000e+00,-4.310772e+06,0.000000e+00,3.847011e+06,-9.313226e-10,9.313226e-10,-2.683179e+06,-4.310772e+06,3.847012e+06


In [39]:
result = train_result(exp107)
print(result.score)

3.0313250900367783
